In [65]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import torch.autograd as autograd
from torch.nn import init
import torch.nn.utils.rnn 
import datetime
import operator
import codecs
from datasets import load_dataset
import unicodedata
import string
import re

np.random.seed(0)

In [3]:
raw_dataset = load_dataset("conv_ai_2")

Reusing dataset conv_ai_2 (C:\Users\justi\.cache\huggingface\datasets\conv_ai_2\conv_ai_2\1.0.0\11d600ddce66bb9d07ca50d1b55b488145ef0d5d0206168c32f1043677875865)


In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialog_id', 'dialog', 'bot_profile', 'user_profile', 'eval_score', 'profile_match'],
        num_rows: 3495
    })
})

In [11]:
raw_train_dataset = raw_dataset["train"]
raw_dialog_dataset = raw_train_dataset["dialog"]
raw_eval_dataset = raw_train_dataset["eval_score"]

In [13]:
raw_dialog_dataset[0]

[{'id': 0,
  'sender': 'participant2',
  'text': 'I love iphone! i just bought new iphone!',
  'sender_class': 'Bot'},
 {'id': 1,
  'sender': 'participant1',
  'text': "Thats good for you, i'm not very into new tech",
  'sender_class': 'Human'},
 {'id': 2,
  'sender': 'participant2',
  'text': 'I am a college student and i am a college student',
  'sender_class': 'Bot'},
 {'id': 3,
  'sender': 'participant1',
  'text': 'I am go to gym and live on donations',
  'sender_class': 'Human'},
 {'id': 4,
  'sender': 'participant2',
  'text': 'I am a vegan and i am in the midwest',
  'sender_class': 'Bot'},
 {'id': 5,
  'sender': 'participant1',
  'text': 'So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat',
  'sender_class': 'Human'},
 {'id': 6,
  'sender': 'participant2',
  'text': 'I would not mind having them in the office that would be hard for me',
  'sender_class': 'Bot'},
 {'id': 7,
  'sender': 'participant1',
  'text': 'Dogs or vegan in offic

In [26]:
type(raw_train_dataset)

datasets.arrow_dataset.Dataset

In [33]:
raw_eval_dataset[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [40]:
# Dialog pairs consist of [context, response, label]
def get_dialog_pairs(dataset):
    dataset_length = len(dataset)
    dialog_pairs = []
    for i in range(dataset_length - 2000):
        conv_length = len(dataset[i])
        for j in range(1,conv_length):
            inputLine = dataset[i][j-1]['text'].strip() 
            if not isinstance(inputLine, str): 
                inputLine = 'None'
            targetLine = dataset[i][j]['text'].strip() 
            if not isinstance(targetLine, str): 
                targetLine = 'None'
            if isinstance(inputLine, str) and isinstance(targetLine, str):
                if raw_eval_dataset[i] > 1:
                    label = 1
                else:
                    label = 0
                dialog_pairs.append([inputLine, targetLine, label])
                
    print(dialog_pairs)
    return dialog_pairs

In [48]:
# Get list of individual sentences
def get_dialog(dataset):
    dataset_length = len(dataset)
    dialog = []
    for i in range(dataset_length - 2000):
        conv_length = len(dataset[i])
        for j in range(1,conv_length):
            inputLine = dataset[i][j-1]['text'].strip() 
            if not isinstance(inputLine, str): 
                inputLine = 'None'
            dialog.append([inputLine])

    return dialog

In [41]:
dialog_pairs = get_dialog_pairs(raw_dialog_dataset)

[['I love iphone! i just bought new iphone!', "Thats good for you, i'm not very into new tech", 0], ["Thats good for you, i'm not very into new tech", 'I am a college student and i am a college student', 0], ['I am a college student and i am a college student', 'I am go to gym and live on donations', 0], ['I am go to gym and live on donations', 'I am a vegan and i am in the midwest', 0], ['I am a vegan and i am in the midwest', 'So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat', 0], ['So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat', 'I would not mind having them in the office that would be hard for me', 0], ['I would not mind having them in the office that would be hard for me', 'Dogs or vegan in office?', 0], ['Dogs or vegan in office?', 'I am a vegetarian so i am vegan', 0], ['I am a vegetarian so i am vegan', 'Strange answer', 0], ['Strange answer', 'I guess i just do not know what to ps4 e

In [42]:
dialog_pairs[0][0]

'I love iphone! i just bought new iphone!'

In [49]:
dialog_indiv = get_dialog(raw_dialog_dataset)

In [50]:
dialog_indiv[:10]

[['I love iphone! i just bought new iphone!'],
 ["Thats good for you, i'm not very into new tech"],
 ['I am a college student and i am a college student'],
 ['I am go to gym and live on donations'],
 ['I am a vegan and i am in the midwest'],
 ['So vegan... i have dogs maybe i should told then that they may eat cheap salads insted of meat'],
 ['I would not mind having them in the office that would be hard for me'],
 ['Dogs or vegan in office?'],
 ['I am a vegetarian so i am vegan'],
 ['Strange answer']]

In [57]:
MAX_LENGTH = 10
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [74]:
def normalizeString(s):
    if isinstance(s, str): 
        s = unicodeToAscii(s.lower().strip())
        s = re.sub(r"([.!?])", r" \1", s)
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
        s = re.sub(r"\s+", r" ", s).strip()
    return s

In [22]:
def prepareData(dataset, name):
    print("Start preparing training data ...")
    voc, pairs = readVocs(dataset, name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

In [ ]:
d_name = "convai2_set"
model_name = "seq2seq-retrieval"
# Load/Assemble voc and pairs
voc, pairs = prepareData(raw_dialog_dataset, d_name)

In [87]:
def create_dataframe(csvfile):
    dataframe = pd.read_csv(csvfile)
    return dataframe

def shuffle_dataframe(dataframe):
    dataframe.reindex(np.random.permutation(dataframe.index))

def create_vocab(dialog):
    vocab = []
    word_freq = {}
    
    for sentence in dialog:
        
        train_words = str(sentence).split(" ")
        
        for word in train_words:
          
            if word.lower() not in vocab:
                vocab.append(word.lower())         
                       
            if word.lower() not in word_freq:
                word_freq[word.lower()] = 1
            else:
                #type(word)
                word_freq[word] += 1
    
    word_freq_sorted = sorted(word_freq.items(), key=lambda item: item[1], reverse=True)
    vocab = ["<UNK>"] + [pair[0] for pair in word_freq_sorted]
    
    return vocab


def create_word_to_id(vocab):             
    word_to_id = {word: id for id, word in enumerate(vocab)}
    
    return word_to_id


def create_id_to_vec(word_to_id, glovefile): 
    lines = open(glovefile, 'r', encoding='utf-8').readlines()
    id_to_vec = {}
    vector = None
    
    for line in lines:
        word = line.split()[0]
        vector = np.array(line.split()[1:], dtype='float32') #32
        
        if word in word_to_id:
            id_to_vec[word_to_id[word]] = torch.FloatTensor(torch.from_numpy(vector))
            
    for word, id in word_to_id.items(): 
        if word_to_id[word] not in id_to_vec:
            v = np.zeros(*vector.shape, dtype='float32')
            v[:] = np.random.randn(*v.shape)*0.01
            id_to_vec[word_to_id[word]] = torch.FloatTensor(torch.from_numpy(v))
            
    embedding_dim = id_to_vec[0].shape[0]
    
    return id_to_vec, embedding_dim


def load_ids_and_labels(pair, word_to_id):
    context_ids = []
    response_ids = []

    context_cell = pair[0]
    response_cell = pair[1]
    label_cell = pair[2]

    max_context_len = 160
    
    context_words = context_cell.split()
    if len(context_words) > max_context_len:
        context_words = context_words[:max_context_len]
    for word in context_words:
        if word in word_to_id:
            context_ids.append(word_to_id[word])
        else: 
            context_ids.append(0) #UNK
    
    response_words = response_cell.split()
    for word in response_words:
        if word in word_to_id:
            response_ids.append(word_to_id[word])
        else: 
            response_ids.append(0)
    
    label = np.array(label_cell).astype(np.float32)

    return context_ids, response_ids, label

In [88]:
create_vocab(dialog_indiv)

['<UNK>',
 "['i",
 'i',
 "?']",
 'you',
 'a',
 '.',
 ".']",
 'am',
 'to',
 'do',
 'like',
 'is',
 'are',
 'and',
 'have',
 'not',
 'the',
 'my',
 'what',
 'm',
 'in',
 'of',
 'love',
 'it',
 "['what",
 'your',
 's',
 'how',
 'but',
 'that',
 '!',
 "['do",
 'for',
 "!']",
 't',
 'about',
 "['that",
 'good',
 "['hi",
 '?',
 'at',
 'sure',
 'from',
 'me',
 'with',
 'don',
 'work',
 "['yes",
 'doing',
 'be',
 'so',
 'just',
 "['no",
 'favorite',
 'go',
 'old',
 "['hello",
 'very',
 'sorry',
 "['how",
 "['you",
 'know',
 'any',
 'live',
 "['are",
 'play',
 'too',
 'great',
 'cool',
 'music',
 'teacher',
 'think',
 "['hey",
 'going',
 'time',
 'on',
 'eat',
 'up',
 'lot',
 'fan',
 'job',
 "['where",
 'dog',
 'little',
 "['hi']",
 "you']",
 'was',
 'there',
 'name',
 'can',
 'person',
 'here',
 'day',
 'see',
 'school',
 'food',
 'all',
 "['oh",
 'will',
 'an',
 'want',
 "['my",
 "['']",
 'dogs',
 'they',
 "['nice",
 "too']",
 'huge',
 'really',
 'living',
 'where',
 'get',
 "['why",
 "['hell

In [82]:
dialog_indiv = [[normalizeString(s) for s in l] for l in dialog_indiv]

In [76]:
dialog_pairs = [[normalizeString(s) for s in l] for l in dialog_pairs]

In [ ]:
test_load = load_ids_and_labels()

In [89]:
class Encoder(nn.Module):

    def __init__(self, 
            emb_size, 
            hidden_size, 
            vocab_size, 
            p_dropout): 
    
            super(Encoder, self).__init__()
             
            self.emb_size = emb_size
            self.hidden_size = hidden_size
            self.vocab_size = vocab_size
            self.p_dropout = p_dropout
       
            self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
            self.lstm = nn.LSTM(self.emb_size, self.hidden_size)
            self.dropout_layer = nn.Dropout(self.p_dropout) 

            self.init_weights()
             
    def init_weights(self):
        init.uniform(self.lstm.weight_ih_l0, a = -0.01, b = 0.01)
        init.orthogonal(self.lstm.weight_hh_l0)
        self.lstm.weight_ih_l0.requires_grad = True
        self.lstm.weight_hh_l0.requires_grad = True
        
        embedding_weights = torch.FloatTensor(self.vocab_size, self.emb_size)
            
        for id, vec in id_to_vec.items():
            embedding_weights[id] = vec
        
        self.embedding.weight = nn.Parameter(embedding_weights, requires_grad = True)
            
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        _, (last_hidden, _) = self.lstm(embeddings) #dimensions: (num_layers * num_directions x batch_size x hidden_size)
        last_hidden = self.dropout_layer(last_hidden[-1])#access last lstm layer, dimensions: (batch_size x hidden_size)

        return last_hidden

    
class DualEncoder(nn.Module):
     
    def __init__(self, encoder):
        super(DualEncoder, self).__init__()
        self.encoder = encoder
        self.hidden_size = self.encoder.hidden_size
        M = torch.FloatTensor(self.hidden_size, self.hidden_size)     
        init.xavier_normal(M)
        self.M = nn.Parameter(M, requires_grad = True)

    def forward(self, context_tensor, response_tensor):
        
        context_last_hidden = self.encoder(context_tensor) #dimensions: (batch_size x hidden_size)
        response_last_hidden = self.encoder(response_tensor) #dimensions: (batch_size x hidden_size)
        
        #context = context_last_hidden.mm(self.M).cuda()
        context = context_last_hidden.mm(self.M) #dimensions: (batch_size x hidden_size)
        context = context.view(-1, 1, self.hidden_size) #dimensions: (batch_size x 1 x hidden_size)
        
        response = response_last_hidden.view(-1, self.hidden_size, 1) #dimensions: (batch_size x hidden_size x 1)
        
        #score = torch.bmm(context, response).view(-1, 1).cuda()
        score = torch.bmm(context, response).view(-1, 1) #dimensions: (batch_size x 1 x 1) and lastly --> (batch_size x 1)

        return score

In [90]:
def creating_variables(num_training_examples, num_validation_examples, embedding_dim):

    print(str(datetime.datetime.now()).split('.')[0], "Creating variables for training and validation...")

    training_dataframe = create_dataframe('dataset/MAIN FILES/training_%d.csv' %num_training_examples)
    vocab = create_vocab(training_dataframe)
    word_to_id = create_word_to_id(vocab)
    id_to_vec, emb_dim = create_id_to_vec(word_to_id, 'dataset/GloVe/glove.6B.%dd.txt' %embedding_dim)

    validation_dataframe = create_dataframe('dataset/MAIN FILES/validation_%d.csv' %num_validation_examples)

    print(str(datetime.datetime.now()).split('.')[0], "Variables created.\n")
    
    return training_dataframe, vocab, word_to_id, id_to_vec, emb_dim, validation_dataframe

In [ ]:
def creating_variables_r(num_training_examples, num_validation_examples, embedding_dim):

    print(str(datetime.datetime.now()).split('.')[0], "Creating variables for training and validation...")

    training_dataframe = create_dataframe('dataset/MAIN FILES/training_%d.csv' %num_training_examples)
    vocab = create_vocab(training_dataframe)
    word_to_id = create_word_to_id(vocab)
    id_to_vec, emb_dim = create_id_to_vec(word_to_id, 'dataset/GloVe/glove.6B.%dd.txt' %embedding_dim)

    validation_dataframe = create_dataframe('dataset/MAIN FILES/validation_%d.csv' %num_validation_examples)

    print(str(datetime.datetime.now()).split('.')[0], "Variables created.\n")
    
    return training_dataframe, vocab, word_to_id, id_to_vec, emb_dim, validation_dataframe

In [6]:
def creating_model(hidden_size, p_dropout):

    print(str(datetime.datetime.now()).split('.')[0], "Calling model...")

    encoder = Encoder(
            emb_size = emb_dim,
            hidden_size = hidden_size,
            vocab_size = len(vocab),
            p_dropout = p_dropout)

    dual_encoder = DualEncoder(encoder)

    print(str(datetime.datetime.now()).split('.')[0], "Model created.\n")
    print(dual_encoder)
    
    return encoder, dual_encoder

In [7]:
def increase_count(correct_count, score, label):
    if ((score.data[0][0] >= 0.5) and (label.data[0][0] == 1.0)) or ((score.data[0][0] < 0.5) and (label.data[0][0]  == 0.0)):
       correct_count +=1  
   
    return correct_count

def get_accuracy(correct_count, dataframe):
    accuracy = correct_count/(len(dataframe))
        
    return accuracy

In [30]:
def train_model(learning_rate, l2_penalty, epochs): 
    print(str(datetime.datetime.now()).split('.')[0], "Starting training and validation...\n")
    print("====================Data and Hyperparameter Overview====================\n")
    print("Number of training examples: %d, Number of validation examples: %d" %(len(training_dataframe), len(validation_dataframe)))
    print("Learning rate: %.5f, Embedding Dimension: %d, Hidden Size: %d, Dropout: %.2f, L2:%.10f\n" %(learning_rate, emb_dim, encoder.hidden_size, encoder.p_dropout, l2_penalty))
    print("================================Results...==============================\n")

    optimizer = torch.optim.Adam(dual_encoder.parameters(), lr = learning_rate, weight_decay = l2_penalty)
    loss_func = torch.nn.BCEWithLogitsLoss()
    #loss_func.cuda()
    best_validation_accuracy = 0.0
     
    for epoch in range(epochs): 
                     
            shuffle_dataframe(training_dataframe)
            sum_loss_training = 0.0
            training_correct_count = 0
            dual_encoder.train()

            for index, row in training_dataframe.iterrows():            
            
                context_ids, response_ids, label = load_ids_and_labels(row, word_to_id)
                context = autograd.Variable(torch.LongTensor(context_ids).view(-1,1), requires_grad = False)#.cuda()
                response = autograd.Variable(torch.LongTensor(response_ids).view(-1, 1), requires_grad = False)#.cuda()
                label = autograd.Variable(torch.FloatTensor(torch.from_numpy(np.array(label).reshape(1,1))), requires_grad = False)#.cuda()
                             
                score = dual_encoder(context, response)
                loss = loss_func(score, label)
                sum_loss_training += loss.item() #loss.data[0]
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                training_correct_count = increase_count(training_correct_count, score, label)
                                                    
            training_accuracy = get_accuracy(training_correct_count, training_dataframe)
            #plt.plot(epoch, training_accuracy)
                
            shuffle_dataframe(validation_dataframe)
            
            validation_correct_count = 0
            sum_loss_validation = 0.0
            dual_encoder.eval()

            for index, row in validation_dataframe.iterrows():
                
                context_ids, response_ids, label = load_ids_and_labels(row, word_to_id)
                context = autograd.Variable(torch.LongTensor(context_ids).view(-1,1)) #.cuda()
                response = autograd.Variable(torch.LongTensor(response_ids).view(-1, 1)) #.cuda()
                label = autograd.Variable(torch.FloatTensor(torch.from_numpy(np.array(label).reshape(1,1)))) #.cuda()
                
                score = dual_encoder(context, response)
                loss = loss_func(score, label)
                sum_loss_validation += loss.item() #loss.data[0]
                validation_correct_count = increase_count(validation_correct_count, score, label)
                    
            validation_accuracy = get_accuracy(validation_correct_count, validation_dataframe)
                        
            print(str(datetime.datetime.now()).split('.')[0], 
                  "Epoch: %d/%d" %(epoch,epochs),  
                  "TrainLoss: %.3f" %(sum_loss_training/len(training_dataframe)), 
                  "TrainAccuracy: %.3f" %(training_accuracy), 
                  "ValLoss: %.3f" %(sum_loss_validation/len(validation_dataframe)), 
                  "ValAccuracy: %.3f" %(validation_accuracy))
            
            if validation_accuracy > best_validation_accuracy:
                best_validation_accuracy = validation_accuracy
                torch.save(dual_encoder.state_dict(), 'dataset/models/saved_model_%d_examples.pt' %(len(training_dataframe)))
                print("New best found and saved.")
                
    print(str(datetime.datetime.now()).split('.')[0], "Training and validation epochs finished.")

In [9]:
training_dataframe, vocab, word_to_id, id_to_vec, emb_dim, validation_dataframe = creating_variables(num_training_examples = 10000, 
                                                                                                     embedding_dim = 50, 
                                                                                                     num_validation_examples = 1000)

2021-11-07 13:35:07 Creating variables for training and validation...
2021-11-07 13:35:30 Variables created.



In [10]:
encoder, dual_encoder = creating_model(hidden_size = 50, 
                                       p_dropout = 0.85)

#encoder.cuda()
#dual_encoder.cuda

for name, param in dual_encoder.named_parameters():
    if param.requires_grad:
        print(name)

2021-11-07 13:35:30 Calling model...
2021-11-07 13:35:30 Model created.

DualEncoder(
  (encoder): Encoder(
    (embedding): Embedding(29997, 50)
    (lstm): LSTM(50, 50)
    (dropout_layer): Dropout(p=0.85, inplace=False)
  )
)
M
encoder.embedding.weight
encoder.lstm.weight_ih_l0
encoder.lstm.weight_hh_l0
encoder.lstm.bias_ih_l0
encoder.lstm.bias_hh_l0


<ipython-input-4-53be591e4b9f>:23: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  init.uniform(self.lstm.weight_ih_l0, a = -0.01, b = 0.01)
<ipython-input-4-53be591e4b9f>:24: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  init.orthogonal(self.lstm.weight_hh_l0)
<ipython-input-4-53be591e4b9f>:50: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(M)


In [32]:
train_model(learning_rate = 0.0001, 
            l2_penalty = 0.0001,
            epochs = 100)

2021-11-08 18:14:31 Starting training and validation...

====================Data and Hyperparameter Overview====================

Number of training examples: 10000, Number of validation examples: 1000
Learning rate: 0.00010, Embedding Dimension: 50, Hidden Size: 50, Dropout: 0.85, L2:0.0001000000

================================Results...==============================

2021-11-08 18:28:52 Epoch: 0/100 TrainLoss: 0.693 TrainAccuracy: 0.501 ValLoss: 0.693 ValAccuracy: 0.502
New best found and saved.
2021-11-08 18:46:49 Epoch: 1/100 TrainLoss: 0.692 TrainAccuracy: 0.501 ValLoss: 0.693 ValAccuracy: 0.502
2021-11-08 19:01:29 Epoch: 2/100 TrainLoss: 0.693 TrainAccuracy: 0.501 ValLoss: 0.693 ValAccuracy: 0.502
2021-11-08 19:12:25 Epoch: 3/100 TrainLoss: 0.691 TrainAccuracy: 0.503 ValLoss: 0.694 ValAccuracy: 0.502
2021-11-08 19:23:13 Epoch: 4/100 TrainLoss: 0.689 TrainAccuracy: 0.507 ValLoss: 0.695 ValAccuracy: 0.507
New best found and saved.
2021-11-08 19:32:50 Epoch: 5/100 TrainLoss: 0.68

In [ ]:

dual_encoder.load_state_dict(torch.load('dataset/models/saved_model_10000_examples.pt'))

dual_encoder.eval()

In [ ]:
test_dataframe_same_structure = pd.read_csv('testing_same_structure_1000.csv')

In [ ]:
def testing_same_structure():
    
    test_correct_count = 0

    for index, row in test_dataframe_same_structure.iterrows():

        context_ids, response_ids, label = load_ids_and_labels(row, word_to_id)
        context = autograd.Variable(torch.LongTensor(context_ids).view(-1,1)) #.cuda()
        response = autograd.Variable(torch.LongTensor(response_ids).view(-1, 1)) #.cuda()
        label = autograd.Variable(torch.FloatTensor(torch.from_numpy(np.array(label).reshape(1,1)))) #.cuda()

        score = dual_encoder(context, response)
        test_correct_count = increase_count(test_correct_count, score, label)

    test_accuracy = get_accuracy(test_correct_count, test_dataframe_same_structure)
    
    return test_accuracy

In [ ]:
test_accuracy = testing_same_structure()
print("Test accuracy for %d training examples and %d test examples: %.2f" %(len(training_dataframe),len(test_dataframe_same_structure),test_accuracy))